In [14]:
from selenium.webdriver.firefox.firefox_binary import FirefoxBinary
from selenium.webdriver.common.keys import Keys
from selenium import webdriver
from collections import defaultdict

In [28]:
browser = webdriver.Firefox()

# 신체 정보 추출

In [16]:
class Body:
    availableTabs=defaultdict() # 현재페이지에 이용가능한 탭과 그 링크
    descrips=defaultdict()
    name="" #신체이름
    part="" #해당부위
    disease=""#해당 질환
    medDep="" #해당 진료과
    related_diseases=[] # 관련질환
    related_interventions=[] # 관련시술/수술/검사

In [17]:
# 인체 정보 총 272개
bodyList = []
st = "http://www.amc.seoul.kr/asan/healthinfo/body/bodyDetail.do?bodyId="

for i in range(272):
        stnew = st + str(i+1)
        bodyList.append(stnew)

In [18]:
#http://www.amc.seoul.kr/asan/healthinfo/body/bodyDetail.do?
#bodyId=150&   신체 id
#tabIndex=1&   탭 번호(0: 설명, 1: 관련질환, 3: 관련시술/수술/검사 인듯)
#pageIndex=1   페이지 번호(결과가 여러개일 경우 페이지 번호도 여럿)

def body_extract(bodyList):
    bodies = []
    for address in bodyList: # http://www.amc.seoul.kr/asan/healthinfo/body/bodyDetail.do?bodyId=1
        
        browser.get(address)
        
        if browser.find_elements_by_class_name("contTitle")[0].text == "()":
            continue
        
        body = Body()
        
        tableIndex= ["1", "3"]

        # 신체 이름
        name = ""
        summaryBox = browser.find_elements_by_class_name("contBox")
        title = summaryBox[0].find_elements_by_class_name("contTitle")
        body.name = title[0].text
        # 신체 이름 - 끝

        # 부위명/질환명/진료과
        summaries = summaryBox[0].find_elements_by_tag_name("dd")
        if len(summaries) >= 1:
            body.part = summaries[0].text
            if len(summaries) >= 2:
                body.disease = summaries[1].text
                if len(summaries) == 3:
                    body.medDep = summaries[2].text
        # 부위명/질환명/진료과 - 끝

        # 본문 정리 -> 본문 정리는 기본 url로 진행 가능
        ds = {}
        descOuter = browser.find_elements_by_class_name("contDescription")
        definitions = descOuter[0].find_elements_by_tag_name("dt")
        descriptions = descOuter[0].find_elements_by_tag_name("dd")
        if len(definitions) == len(descriptions):
            for defi, des in zip(definitions, descriptions):
                ds[defi.text] = des.text
            body.descrips = ds
        else:
            print("정의와 설명의 개수가 다름")
        # 본문 정리 - 끝

        # 다음 탭으로 이동
        nextUrl = address + "&tabIndex=" + tableIndex[0]
        browser.get(nextUrl)

        # 페이지 길이 파악
        pageOuter = browser.find_elements_by_class_name("numPagingSec")
        if len(pageOuter) >= 1:
            pages = pageOuter[0].find_elements_by_tag_name("a")
            pageLen = int(pages[-1].text) # 전체 페이지 길이
        # 페이지 길이 파악 - 끝

        urlToExplore = [] # 페이지 수에 따라 조사해야 할 url 리스트
        for i in range(pageLen):
                nextUrl = address + "&tabIndex=" + tableIndex[0] + "&pageIndex=" + str(i+1)
                urlToExplore.append(nextUrl)
                
                
        #관련 질병 추출
        dis = []
        for url in urlToExplore:
            browser.get(url)
            diseasesOuter = browser.find_elements_by_class_name("listCont")
            if len(diseasesOuter) >= 1:
                diseases = diseasesOuter[0].find_elements_by_class_name("contTitle")
                for disease in diseases:
                    dis.append(disease.text)
        body.related_diseases = dis
        # 관련 질병 추출 -끝
        
        # 다음 탭으로 이동
        nextUrl = address + "&tabIndex=" + tableIndex[1]
        browser.get(nextUrl)

        # 페이지 길이 파악
        pageOuter = browser.find_elements_by_class_name("numPagingSec")
        if len(pageOuter) >= 1:
            pages = pageOuter[0].find_elements_by_tag_name("a")
            pageLen = int(pages[-1].text) # 전체 페이지 길이
        # 페이지 길이 파악 - 끝

        urlToExplore = [] # 페이지 수에 따라 조사해야 할 url 리스트
        for i in range(pageLen):
                nextUrl = address + "&tabIndex=" + tableIndex[1] + "&pageIndex=" + str(i+1)
                urlToExplore.append(nextUrl)
                
        #관련 검사/시술/수술
        inter = []
        for url in urlToExplore:
            browser.get(url)
            interventionsOuter = browser.find_elements_by_class_name("listCont")
            if len(interventionsOuter) >= 1:
                interventions = interventionsOuter[0].find_elements_by_class_name("contTitle")
                for intervention in interventions:
                    inter.append(intervention.text)
        body.related_interventions = inter
        #관련 검사/시술/수술 - 끝
        
        bodies.append(body)
        print(body.name, "완료. 현재 완료한 객체: ", len(bodies))
    return bodies    

In [19]:
bodies = body_extract(bodyList)

간(Liver) 완료. 현재 완료한 객체:  1
갈비뼈(Rib) 완료. 현재 완료한 객체:  2
갑상선(Thyroid) 완료. 현재 완료한 객체:  3
겨드랑이(Axilla) 완료. 현재 완료한 객체:  4
견갑골(Scapula) 완료. 현재 완료한 객체:  5
고환(Testis) 완료. 현재 완료한 객체:  6
골반(Pelvis) 완료. 현재 완료한 객체:  7
골수(Bone marrow) 완료. 현재 완료한 객체:  8
관절(Joint) 완료. 현재 완료한 객체:  9
괄약근(Sphincter) 완료. 현재 완료한 객체:  10
귀(Ear) 완료. 현재 완료한 객체:  11
근육(Muscle) 완료. 현재 완료한 객체:  12
기관지(Bronchus) 완료. 현재 완료한 객체:  13
난소(Ovary) 완료. 현재 완료한 객체:  14
뇌(Brain) 완료. 현재 완료한 객체:  15
뇌하수체(Pituitary gland) 완료. 현재 완료한 객체:  16
눈(Eye) 완료. 현재 완료한 객체:  17
뉴런(Neuron) 완료. 현재 완료한 객체:  18
담낭(Gallbladder) 완료. 현재 완료한 객체:  19
대뇌(Cerebrum) 완료. 현재 완료한 객체:  20
대장(Large intestine) 완료. 현재 완료한 객체:  21
땀샘(Sweat gland) 완료. 현재 완료한 객체:  22
림프(Lymph) 완료. 현재 완료한 객체:  23
말초신경계(Peripheral Nerve System) 완료. 현재 완료한 객체:  24
맹장(Cecum) 완료. 현재 완료한 객체:  25
충수(Appendix) 완료. 현재 완료한 객체:  26
무릎(Knee) 완료. 현재 완료한 객체:  27
발(Foot) 완료. 현재 완료한 객체:  28
방광(Bladder) 완료. 현재 완료한 객체:  29
복강(Abdominal cavity) 완료. 현재 완료한 객체:  30
부신(Adrenal gland) 완료. 현재 완료한 객체:  31
비장(Spleen) 완

안구(Eyeball) 완료. 현재 완료한 객체:  229
안와(Orbit ) 완료. 현재 완료한 객체:  230
각막(Cornea ) 완료. 현재 완료한 객체:  231
공막(Sclera) 완료. 현재 완료한 객체:  232
포도막(Uvea ) 완료. 현재 완료한 객체:  233
홍채(Iris) 완료. 현재 완료한 객체:  234
모양체(Ciliary body) 완료. 현재 완료한 객체:  235
맥락막(Choroid ) 완료. 현재 완료한 객체:  236
수정체(Lens) 완료. 현재 완료한 객체:  237
유리체(Vitreous body) 완료. 현재 완료한 객체:  238
황반(Macula lutea) 완료. 현재 완료한 객체:  239
기관(Trachea) 완료. 현재 완료한 객체:  240
세기관지(Bronchiole ) 완료. 현재 완료한 객체:  241
진피(Dermis ) 완료. 현재 완료한 객체:  242
귀두(Glans ) 완료. 현재 완료한 객체:  243
부고환(Epididymis ) 완료. 현재 완료한 객체:  244
음낭(Scrotum ) 완료. 현재 완료한 객체:  245
정낭(Seminal vesicle ) 완료. 현재 완료한 객체:  246
대음순(Labium major) 완료. 현재 완료한 객체:  247
소음순(Labium minor) 완료. 현재 완료한 객체:  248
음핵(Clitoris ) 완료. 현재 완료한 객체:  249
처녀막(Hymen) 완료. 현재 완료한 객체:  250
유두(Nipple ) 완료. 현재 완료한 객체:  251
유륜(Areola ) 완료. 현재 완료한 객체:  252
난관(Uterine tube ) 완료. 현재 완료한 객체:  253
자궁내막(Endometrium) 완료. 현재 완료한 객체:  254
양막(Amnion ) 완료. 현재 완료한 객체:  255
융모막융모(Chorionic villus) 완료. 현재 완료한 객체:  256
황체(Corpus luteum) 완료. 현재 완료한 객체:  2

In [ ]:
# http://www.amc.seoul.kr/asan/healthinfo/body/bodyDetail.do?bodyId=
# 확인용 테스트
i = 100 # 신체정보 번호 0 ~ 271 -> 272개
print("신체명: ", bodies[i].name)
print("부위명: ",bodies[i].part)
print("관련질환: ", bodies[i].disease)
print("관련 진료과: ", bodies[i].medDep)
print()
print("관련 설명: ")
for k, v in bodies[i].descrips.items():
    print(k)
    print(v)
print()
print("관련 질환명: ")
for d in bodies[i].related_diseases:
    print(d)
print()
print("관련 시술/수술/검사 명: ")
for i in bodies[i].related_interventions:
    print(i)

In [ ]:
txt= ""
for body in bodies:
    txt = txt + "================================= \n"
    txt = txt + "신체명: " + body.name + "\n"
    if len(body.part) >= 1:
        txt = txt + "부위명: " + body.part + "\n"
    if len(body.disease) >= 1:
        txt = txt + "관련질환: " + body.disease + "\n"
    if len(body.medDep) >= 1:
        txt = txt + "관련 진료과: " + body.medDep + "\n"
    if len(body.descrips) >= 1:
        for k, v in body.descrips.items():
            txt = txt + k + "\n"
            txt = txt + v + "\n"
    if len(body.related_diseases) >= 1:
        txt = txt + "연관된 질환" + "\n"
        for d in body.related_diseases:
            txt = txt + d + "\n"
    if len(body.related_interventions) >= 1:
        txt = txt + "연관된 시술/수술/검사" + "\n"
        for d in body.related_interventions:
            txt = txt + d + "\n"
    txt = txt + "================================= \n"
#print(txt)

with open("./output/whole/bodies.txt", "w") as text_file:
    text_file.write(txt)

In [ ]:
txt= ""
for body in bodies:
    txt = txt + "신체명: " + body.name + "\n"
    if len(body.descrips) >= 1:
        for k, v in body.descrips.items():
            txt = txt + k + "\n"
            txt = txt + v + "\n"

with open("./output/whole/bodies_description.txt", "w") as text_file:
    text_file.write(txt)

In [ ]:
# 항목별 txt 파일
for body in bodies:
    txt= ""
    txt = txt + "신체명: " + body.name + "\n"
    if len(body.descrips) >= 1:
        for k, v in body.descrips.items():
            txt = txt + k + "\n"
            txt = txt + v + "\n"
    with open("./output/body_"+ body.name +".txt", "w") as text_file:
        text_file.write(txt)

# 질환 정보 추출

In [36]:
class Disease:
    availableTabs=defaultdict() # 현재페이지에 이용가능한 탭과 그 링크
    descrips=defaultdict() # 질환 설명
    healthStory=defaultdict() # 건강이야기
    name="" #질병이름
    related_symptoms=[] #관련증상
    related_diseases=[] # 관련질환
    medDeps=[] #해당 진료과
    synonym="" # 동의어

In [37]:
# 질환 정보 총 958개  contentId= 31541 - 32499

# www.amc.seoul.kr/asan/healthinfo/disease/diseaseDetail.do?contentId=31541
# http://www.amc.seoul.kr/asan/healthinfo/disease/diseaseDetail.do?contentId=32499

diseaseList = []
st = "http://www.amc.seoul.kr/asan/healthinfo/disease/diseaseDetail.do?contentId="

for i in range(958):
        stnew = st + str(31540+ i+1)
        diseaseList.append(stnew)

In [ ]:
# 중간중간 페이지가 비어있는 것으로 확인
print(diseaseList[-29])

In [38]:
#http://www.amc.seoul.kr/asan/healthinfo/disease/diseaseDetail.do?
#contentId=31541  질환 아이디
#&tabIndex=0  탭 번호 0: 질환설명 / 4:건강이야기
# 질환 정보 총 958개  contentId= 31541 - 32499

def disease_extract(diseaseList):
    diseases = []
    for address in diseaseList: # http://www.amc.seoul.kr/asan/healthinfo/disease/diseaseDetail.do?contentId=31541
        browser.get(address)
        
        if browser.find_elements_by_class_name("contTitle")[0].text == "()":
            continue
        disease = Disease()
        

        tableIndex= ["4"] # 0: 질환설명 / 4:건강이야기 ------> 의료진 탭과 교육자료는 뺌 

        # 질환 이름
        summaryBox = browser.find_elements_by_class_name("contBox")
        title = summaryBox[0].find_elements_by_class_name("contTitle")
        disease.name = title[0].text
        # 신체 이름 - 끝

        # 증상 / 유사 질환 / 진료과 / 동의어
        summaries = summaryBox[0].find_elements_by_tag_name("dd")
        if len(summaries) >= 1:
            symptoms = summaries[0].find_elements_by_tag_name("a")
            symps = []
            for symptom in symptoms:
                symps.append(symptom.text)
            disease.related_symptoms = symps
            if len(summaries) >= 2:
                diss = summaries[1].find_elements_by_tag_name("a")
                rel_dis = []
                for dis in diss:
                    rel_dis.append(dis.text)
                disease.related_diseases = rel_dis
                if len(summaries) == 3:
                    medDeps = summaries[1].find_elements_by_tag_name("a")
                    deps = []
                    for medDep in medDeps:
                        deps.append(medDep.text)
                    disease.medDeps = deps
                    if len(summaries) == 4:
                        disease.synonym = summaries[3].text
        # 증상 / 유사 질환 / 진료과 / 동의어 - 끝
        
        # 본문 정리 -> 본문 정리는 기본 url로 진행 가능
        ds = {}
        descOuter = browser.find_elements_by_class_name("contDescription")
        definitions = descOuter[0].find_elements_by_tag_name("dt")
        descriptions = descOuter[0].find_elements_by_tag_name("dd")
        if len(definitions) == len(descriptions):
            for defi, des in zip(definitions, descriptions):
                ds[defi.text] = des.text
            disease.descrips = ds
        else:
            print("정의와 설명의 개수가 다름")
        # 본문 정리 - 끝

        # 다음 탭으로 이동
        nextUrl = address + "&tabIndex=" + tableIndex[0]
        browser.get(nextUrl)

        # 건강 이야기 추출
        hl = {}
        healthStories = browser.find_elements_by_class_name("listTypeSec2 photoType widthSmall")
        for healthStory in healthStories:
            titles = browser.find_elements_by_tag_name("dt")
            contents = browser.find_elements_by_tag_name("dd")
            if len(titles) == len(contents):
                for titl, cont in zip(titles, contents):
                    hl[titl.text] = cont.text
                disease.healthStory = hl
            else:
                print("소제목과 내용의 개수가 다름")
        # 건강 이야기 추출 -끝
        
        diseases.append(disease)
        print(disease.name, "완료. 현재 완료한 객체: ", len(diseases))
    return diseases    

In [39]:
diseases = disease_extract(diseaseList) 

대상포진(Herpes zoster) 완료. 현재 완료한 객체:  1
백반증(Vitiligo) 완료. 현재 완료한 객체:  2
여드름(Acne) 완료. 현재 완료한 객체:  3
모낭염(Folliculitis) 완료. 현재 완료한 객체:  4
피부암(Skin cancer) 완료. 현재 완료한 객체:  5
곰팡이 감염(Fungal infection) 완료. 현재 완료한 객체:  6
사마귀(Wart) 완료. 현재 완료한 객체:  7
건선(Psoriasis) 완료. 현재 완료한 객체:  8
매복 사랑니(Impacted teeth) 완료. 현재 완료한 객체:  9
근단 주위 농양(Periapical abscess with sinus) 완료. 현재 완료한 객체:  10
치근낭(Radicular cyst) 완료. 현재 완료한 객체:  11
만성 치주염(Chronic periodontitis) 완료. 현재 완료한 객체:  12
턱크기의 주요 이상(Major anomalies of jaw size) 완료. 현재 완료한 객체:  13
턱뼈 성장 이상(Anomalies of the jaw) 완료. 현재 완료한 객체:  14
치열 궁 관계 이상(Anomalies of dental arch relationship) 완료. 현재 완료한 객체:  15
턱관절 장애(Temporomandibular joint disorders) 완료. 현재 완료한 객체:  16
발육성 치성 낭종(Developmental odontogenic cysts) 완료. 현재 완료한 객체:  17
턱의 기타 낭(Other cysts of jaw) 완료. 현재 완료한 객체:  18
안면 골절(Facial bone fracture) 완료. 현재 완료한 객체:  19
대퇴 골절(Fracture of femur) 완료. 현재 완료한 객체:  20
경골과 비골 골절(Fracture of tibia & fibula) 완료. 현재 완료한 객체:  21
원위 요골과 척골 골절(Fracture of distal radius and u

골괴사(Osteonecrosis) 완료. 현재 완료한 객체:  174
바이러스성 뇌수막염(Viral meningitis) 완료. 현재 완료한 객체:  175
병리적 골절(Pathological fracture) 완료. 현재 완료한 객체:  176
박테리아성 뇌수막염(Bacterial meningitis) 완료. 현재 완료한 객체:  177
척추병증(Spondylopathies) 완료. 현재 완료한 객체:  178
급성 위장염(Acute gastroenteritis) 완료. 현재 완료한 객체:  179
위장관 결핵(Gastrointestinal tuberculosis) 완료. 현재 완료한 객체:  180
결핵성 복막염(Tuberculous peritonitis) 완료. 현재 완료한 객체:  181
아킬레스 건염(Achilles' tendonitis) 완료. 현재 완료한 객체:  182
베이커 낭종(Baker's cyst) 완료. 현재 완료한 객체:  183
내반슬(Bow leg) 완료. 현재 완료한 객체:  184
소건막류(Bunionette) 완료. 현재 완료한 객체:  185
요족(Cavus foot) 완료. 현재 완료한 객체:  186
복합 부위 통증 증후군(Complex regional pain syndrome) 완료. 현재 완료한 객체:  187
매독(Syphilis) 완료. 현재 완료한 객체:  188
패혈성 관절염(Septic arthtritis) 완료. 현재 완료한 객체:  189
급성 방광염(Acute cystitis) 완료. 현재 완료한 객체:  190
급성 신우신염(Acute pyelonephritis) 완료. 현재 완료한 객체:  191
듀피트렌 구축(Dupuytren's contracture) 완료. 현재 완료한 객체:  192
내연골종(Enchondroma) 완료. 현재 완료한 객체:  193
경부 림프절염(Cervical lymphadenitis) 완료. 현재 완료한 객체:  194
결핵성 임파선염(Tuberculous lymphade

수면 장애(Sleep Disorder) 완료. 현재 완료한 객체:  344
외상성 경막하 출혈(Traumatic subdural hemorrhage) 완료. 현재 완료한 객체:  345
광장공포증(Agoraphobia) 완료. 현재 완료한 객체:  346
반사회성 인격장애(Antisocial Personality Disorder) 완료. 현재 완료한 객체:  347
실어증(Aphasia) 완료. 현재 완료한 객체:  348
자폐증 / 자폐 스펙트럼 장애(Autism spectrum disorder) 완료. 현재 완료한 객체:  349
각기병(Beriberi) 완료. 현재 완료한 객체:  350
망상 장애(Delusional disorder) 완료. 현재 완료한 객체:  351
해리 장애(Dissociative disorder) 완료. 현재 완료한 객체:  352
변비(Constipation) 완료. 현재 완료한 객체:  353
난독증(Dyslexia) 완료. 현재 완료한 객체:  354
기분부전증(Dysthymia) 완료. 현재 완료한 객체:  355
설인신경통(Glossopharyngeal neuralgia) 완료. 현재 완료한 객체:  356
길랑-바레 증후군(Guillain-Barre Syndrome) 완료. 현재 완료한 객체:  357
변실금(Fecal incontinence) 완료. 현재 완료한 객체:  358
지간 신경종(Interdigital neuroma) 완료. 현재 완료한 객체:  359
인터넷 중독(Internet addiction) 완료. 현재 완료한 객체:  360
얼렌 증후군(Irlen syndrome) 완료. 현재 완료한 객체:  361
지적장애(Intellectual disability) 완료. 현재 완료한 객체:  362
뮌하우젠 증후군(Munchausen's syndrome) 완료. 현재 완료한 객체:  363
골수섬유증(Myelofibrosis) 완료. 현재 완료한 객체:  364
기면증(Narcolepsy) 완료. 현재 완료

냉방병(Air-conditioningitis) 완료. 현재 완료한 객체:  514
만성 피로 증후군(Chronic Fatigue Syndrome) 완료. 현재 완료한 객체:  515
대사 증후군(Metabolic syndrome) 완료. 현재 완료한 객체:  516
새집 증후군(Sick building syndrome) 완료. 현재 완료한 객체:  517
잠수병(Diver's disease) 완료. 현재 완료한 객체:  518
건강염려증(Hypochondriasis) 완료. 현재 완료한 객체:  519
수족냉증(Coldness of hands and feet) 완료. 현재 완료한 객체:  520
유당분해효소결핍증(Lactose intolerance) 완료. 현재 완료한 객체:  521
급성 담낭염(Acute cholecystitis) 완료. 현재 완료한 객체:  522
간성 뇌증(Hepatic encephalopathy) 완료. 현재 완료한 객체:  523
화농성 간농양(Pyogenic liver abscess) 완료. 현재 완료한 객체:  524
경화성 담관염(Sclerosing cholangitis) 완료. 현재 완료한 객체:  525
급성 B형 간염(Acute hepatitis B) 완료. 현재 완료한 객체:  526
급성 C형 간염(Acute hepatitis C) 완료. 현재 완료한 객체:  527
원발성 담즙성 경화증(Primary Biliary cirrhosis) 완료. 현재 완료한 객체:  528
만성 담낭염(Chronic cholecystitis) 완료. 현재 완료한 객체:  529
대장염(Colitis) 완료. 현재 완료한 객체:  530
헬리코박터균 감염(Helicobacter pylori infection) 완료. 현재 완료한 객체:  531
간 혈관종(Hepatic hemangioma) 완료. 현재 완료한 객체:  532
간신 증후군(Hepatorenal syndrome) 완료. 현재 완료한 객체:  533
카롤리병(Caroli’s di

부갑상선기능항진증(Hyperparathyroidism) 완료. 현재 완료한 객체:  690
유관 확장증(Mammary duct ectasia) 완료. 현재 완료한 객체:  691
흉선암(Thymus Cancer) 완료. 현재 완료한 객체:  692
유방의 미세석회화(Microcalcification of breast) 완료. 현재 완료한 객체:  693
갑상선염(Thyroiditis) 완료. 현재 완료한 객체:  694
말단비대증(Acromegaly) 완료. 현재 완료한 객체:  695
항인지질 항체 증후군(Antiphospholipid antibody syndrome) 완료. 현재 완료한 객체:  696
갑상선 여포암(Follicular thyroid cancer) 완료. 현재 완료한 객체:  697
갑상선 기능 항진증(Hyperthyroidism) 완료. 현재 완료한 객체:  698
갑상선 수질암(Medullary thyroid cancer) 완료. 현재 완료한 객체:  699
포르피린증(Porphyria) 완료. 현재 완료한 객체:  700
갑상선 미분화암(Undifferentiated thyroid cancer) 완료. 현재 완료한 객체:  701
급성 류마티스성 열(Acute Rheumatic fever) 완료. 현재 완료한 객체:  702
보툴리즘 중독(Botulism) 완료. 현재 완료한 객체:  703
무른 궤양(Chancroid) 완료. 현재 완료한 객체:  704
혼수(Coma) 완료. 현재 완료한 객체:  705
압궤 손상(Crushing injury) 완료. 현재 완료한 객체:  706
단독(Erysipelas) 완료. 현재 완료한 객체:  707
돌발성 발진(Exanthem subitum) 완료. 현재 완료한 객체:  708
소화관의 이물(Foreign body in alimentary tract) 완료. 현재 완료한 객체:  709
식도의 이물(Foreign body in esophagus) 완료. 현재 완료한 객체:  710
가스 괴

진주종(Cholesteatoma) 완료. 현재 완료한 객체:  847
만성 비염(Chronic rhinitis) 완료. 현재 완료한 객체:  848
메니에르병(Meniere's disease) 완료. 현재 완료한 객체:  849
비용종(Nasal polyp) 완료. 현재 완료한 객체:  850
이루증(Otorrhea) 완료. 현재 완료한 객체:  851
이명(Tinnitus) 완료. 현재 완료한 객체:  852
편도선염(Tonsillitis) 완료. 현재 완료한 객체:  853
인후두염(Laryngopharyngitis) 완료. 현재 완료한 객체:  854
고막염(Myringitis) 완료. 현재 완료한 객체:  855
이하선염(Parotitis) 완료. 현재 완료한 객체:  856
노인성 난청(Presbycusis) 완료. 현재 완료한 객체:  857
타석증(Sialolithiasis) 완료. 현재 완료한 객체:  858
이식편대숙주질환(Graft-versus-host reaction or disease(GVHD)) 완료. 현재 완료한 객체:  859
면역혈소판감소증(Immune thrombocytopenia) 완료. 현재 완료한 객체:  860
감염성 단핵구증(Infectious mononucleosis) 완료. 현재 완료한 객체:  861
원발성 중추신경계 림프종(Primary CNS lymphoma) 완료. 현재 완료한 객체:  862
베게너 육아종증(Wegener granulomatosis) 완료. 현재 완료한 객체:  863
악액질(Cachexia) 완료. 현재 완료한 객체:  864
조혈모세포 이식(Stem cell transplantation) 완료. 현재 완료한 객체:  865
유잉육종(Ewing sarcoma) 완료. 현재 완료한 객체:  866
과오종(Hamartoma) 완료. 현재 완료한 객체:  867
랑거한스세포 조직구증식증(Langerhans cell histiocytosis) 완료. 현재 완료한 객체:  868
폰빌레브란트병(Von

In [ ]:
# http://www.amc.seoul.kr/asan/healthinfo/disease/diseaseDetail.do?contentId=
# 확인용 테스트    질환 정보 0-907  -> 총 908개 
i = 900 
print("질환명: ", diseases[i].name)
print("관련증상: ")
for symptom in diseases[i].related_symptoms:
    print(symptom)
print("관련질환: ")
for dis in diseases[i].related_diseases:
    print(dis)
print("관련 진료과: ")
for medDep in diseases[i].medDeps:
    print(medDep)
print("동의어: ", diseases[i].synonym)
print()
print("관련 설명: ")
for k, v in diseases[i].descrips.items():
    print(k)
    print(v)
print()
print("건강 이야기: ")
for k, v in diseases[i].healthStory.items():
    print(k)
    print(v)

In [ ]:
txt= ""
for disease in diseases:
    txt = txt + "================================= \n"
    txt = txt + "질환명: " + disease.name + "\n"
    if len(disease.related_symptoms) >= 1:
        txt = txt + "관련증상: " + "\n"
        for symptom in disease.related_symptoms:
            txt = txt + symptom + "\n"
    if len(disease.related_diseases) >= 1:
        txt = txt + "관련질환: " + "\n"
        for dis in disease.related_diseases:
            txt = txt + dis + "\n"
    if len(disease.medDeps) >= 1:
        txt = txt + "관련진료과: " + "\n"
        for medDep in disease.medDeps:
            txt = txt + medDep + "\n"
    if len(disease.synonym) >= 1:
        txt = txt + "동의어: " + disease.synonym + "\n"
    if len(disease.descrips) >= 1:
        txt = txt + "관련 설명: " + "\n"
        for k, v in disease.descrips.items():
            txt = txt + k + "\n"
            txt = txt + v + "\n"
    if len(disease.healthStory) >= 1:
        txt = txt + "건강 이야기: " + "\n"
        for k, v in disease.healthStory.items():
            txt = txt + k + "\n"
            txt = txt + v + "\n"
    txt = txt + "================================= \n"

with open("./output/whole/diseases.txt", "w") as text_file:
    text_file.write(txt)


In [ ]:
txt= ""
for disease in diseases:
    txt = txt + "질환명: " + disease.name + "\n"
    if len(disease.descrips) >= 1:
        txt = txt + "관련 설명: " + "\n"
        for k, v in disease.descrips.items():
            txt = txt + k + "\n"
            txt = txt + v + "\n"
    if len(disease.healthStory) >= 1:
        txt = txt + "건강 이야기: " + "\n"
        for k, v in disease.healthStory.items():
            txt = txt + k + "\n"
            txt = txt + v + "\n"

with open("./output/whole/diseases_description.txt", "w") as text_file:
    text_file.write(txt)


In [ ]:
for disease in diseases:
    if (disease.name.find('/') != -1):
        disease.name = disease.name.replace('/','')

In [ ]:
# 항목별 텍스트
for disease in diseases:
    txt= ""
    txt = txt + "질환명: " + disease.name + "\n"
    if len(disease.descrips) >= 1:
        txt = txt + "관련 설명: " + "\n"
        for k, v in disease.descrips.items():
            txt = txt + k + "\n"
            txt = txt + v + "\n"
    if len(disease.healthStory) >= 1:
        txt = txt + "건강 이야기: " + "\n"
        for k, v in disease.healthStory.items():
            txt = txt + k + "\n"
            txt = txt + v + "\n"
    with open("./output/diseases_"+ disease.name +".txt", "w") as text_file:
        text_file.write(txt)


# 검사/시술/수술 정보 추출

In [24]:
class Intervention:
    name="" # 검사 제목
    availableTabs=defaultdict() # 현재페이지에 이용가능한 탭과 그 링크
    related_parts=[] #관련 신체 부위
    related_areas=[] # 관련 질환 분야
    related_interventions=[] #관련검사
    medDeps=[] # 관련진료과
    descrips=defaultdict() # 검사 설명
    related_diseases=[] # 관련질환
    related_bodies=[] # 관련 인체

In [25]:
interventionList = []
st = "http://www.amc.seoul.kr/asan/healthinfo/management/managementDetail.do?managementId="

for i in range(546):
        stnew = st + str(i+1)
        interventionList.append(stnew)

In [26]:
# http://www.amc.seoul.kr/asan/healthinfo/management/managementDetail.do?managementId=1
# http://www.amc.seoul.kr/asan/healthinfo/management/managementDetail.do?managementId=546

#http://www.amc.seoul.kr/asan/healthinfo/management/managementDetail.do?
#managementId=20    검사 아이디
#&tabIndex=3        탭 번호 0: 설명 / 1: 관련 질환 / 3: 관련 인체

# 검사/시술/수술 정보 총 546개  managementId= 1 - 546

def intervention_extract(interventionList):
    interventions = []
    for address in interventionList: # http://www.amc.seoul.kr/asan/healthinfo/management/managementDetail.do?managementId=
        browser.get(address)
        if browser.find_elements_by_class_name("contTitle")[0].text == "()":
            continue
        
        intervention = Intervention()
        tableIndex= ["1", "3"]

        # 신체 이름
        summaryBox = browser.find_elements_by_class_name("contBox")
        title = summaryBox[0].find_elements_by_class_name("contTitle")
        intervention.name = title[0].text
        # 신체 이름 - 끝

        # 부위명/질환명/진료과
        summaries = summaryBox[0].find_elements_by_tag_name("dd")
        
        if len(summaries) >= 1: # 부위
            parts = summaries[0].find_elements_by_tag_name("a")
            prts = []
            for part in parts:
                prts.append(part.text)
            intervention.related_parts = prts
            
            if len(summaries) >= 2: # 질환
                diseases = summaries[1].find_elements_by_tag_name("a")
                diss = []
                for disease in diseases:
                    diss.append(disease.text)
                intervention.related_areas = diss
                
                if len(summaries) == 3: # 검사
                    intervenes = summaries[2].find_elements_by_tag_name("a")
                    intvs = []
                    for intervene in intervenes:
                        intvs.append(intervene.text)
                    intervention.related_interventions = intvs

                    if len(summaries) == 4: # 진료과
                        medDeps = summaries[3].find_elements_by_tag_name("a")
                        mdps = []
                        for medDep in medDeps:
                            mdps.append(medDep.text)
                        intervention.medDeps = mdps
        # 부위명/질환명/진료과 - 끝

        
        # 본문 정리 -> 본문 정리는 기본 url로 진행 가능
        ds = {}
        descOuter = browser.find_elements_by_class_name("contDescription")
        titles = descOuter[0].find_elements_by_tag_name("dt")
        contents = descOuter[0].find_elements_by_tag_name("dd")
        if len(titles) == len(contents):
            for titl, conts in zip(titles, contents):
                ds[titl.text] = conts.text
            intervention.descrips = ds
        else:
            print("정의와 설명의 개수가 다름")
        # 본문 정리 - 끝

        
        # 다음 탭으로 이동
        nextUrl = address + "&tabIndex=" + tableIndex[0]
        browser.get(nextUrl)

        # 페이지 길이 파악
        pageOuter = browser.find_elements_by_class_name("numPagingSec")
        if len(pageOuter) >= 1:
            pages = pageOuter[0].find_elements_by_tag_name("a")
            pageLen = int(pages[-1].text) # 전체 페이지 길이
        # 페이지 길이 파악 - 끝

        urlToExplore = [] # 페이지 수에 따라 조사해야 할 url 리스트
        for i in range(pageLen):
                nextUrl = address + "&tabIndex=" + tableIndex[0] + "&pageIndex=" + str(i+1)
                urlToExplore.append(nextUrl)
                
                
        #관련 질병 추출
        dis = []
        for url in urlToExplore:
            browser.get(url)
            diseasesOuter = browser.find_elements_by_class_name("listCont")
            if len(diseasesOuter) >= 1:
                diseases = diseasesOuter[0].find_elements_by_class_name("contTitle")
                for disease in diseases:
                    dis.append(disease.text)
        intervention.related_diseases = dis
        # 관련 질병 추출 -끝
        
        # 다음 탭으로 이동
        nextUrl = address + "&tabIndex=" + tableIndex[1]
        browser.get(nextUrl)

        # 페이지 길이 파악
        pageOuter = browser.find_elements_by_class_name("numPagingSec")
        if len(pageOuter) >= 1:
            pages = pageOuter[0].find_elements_by_tag_name("a")
            pageLen = int(pages[-1].text) # 전체 페이지 길이
        # 페이지 길이 파악 - 끝

        urlToExplore = [] # 페이지 수에 따라 조사해야 할 url 리스트
        for i in range(pageLen):
                nextUrl = address + "&tabIndex=" + tableIndex[1] + "&pageIndex=" + str(i+1)
                urlToExplore.append(nextUrl)
                
        # 관련 신체 부위 추출
        bdy = []
        for url in urlToExplore:
            browser.get(url)
            bodiesOuter = browser.find_elements_by_class_name("listCont")
            if len(bodiesOuter) >= 1:
                bodies = bodiesOuter[0].find_elements_by_class_name("contTitle")
                for body in bodies:
                    bdy.append(body.text)
        intervention.related_interventions = bdy
        #관련 신체 부위 추출 - 끝
        
        interventions.append(intervention)
        print(intervention.name, "완료. 현재 완료한 객체: ", len(interventions))
    return interventions

In [29]:
interventions = intervention_extract(interventionList) 

경피적 간담도조영술(Percutaneous transhepatic cholangiography) 완료. 현재 완료한 객체:  1
혈당 검사(Blood sugar test) 완료. 현재 완료한 객체:  2
청력 검사(Audiometry) 완료. 현재 완료한 객체:  3
초음파 검사(Echography) 완료. 현재 완료한 객체:  4
간 초음파 검사(Liver Ultrasonography) 완료. 현재 완료한 객체:  5
혈관조영술(Angiography) 완료. 현재 완료한 객체:  6
관절조영술(Arthrography) 완료. 현재 완료한 객체:  7
기관지조영술(Broncography) 완료. 현재 완료한 객체:  8
담낭조영술(Cholecystography) 완료. 현재 완료한 객체:  9
담도조영술(Cholangiography) 완료. 현재 완료한 객체:  10
방광요도조영술(Cystourethrography) 완료. 현재 완료한 객체:  11
배뇨성 방광요도조영술(Voiding cystourethrography) 완료. 현재 완료한 객체:  12
식도조영술(Esophagography) 완료. 현재 완료한 객체:  13
십이지장조영술(Duodenography) 완료. 현재 완료한 객체:  14
양수조영술(Amniography) 완료. 현재 완료한 객체:  15
역행성 신우조영술(Retrograde Pyelography ) 완료. 현재 완료한 객체:  16
유방촬영술(Mammography) 완료. 현재 완료한 객체:  17
정관정낭조영술(Vesiculography) 완료. 현재 완료한 객체:  18
정맥신우조영술(Intravenous Pyelography) 완료. 현재 완료한 객체:  19
질조영술(Vaginography ) 완료. 현재 완료한 객체:  20
척수조영술(Myelography) 완료. 현재 완료한 객체:  21
췌장관조영술(Pancreatography) 완료. 현재 완료한 객체:  22
자기공명영상검사(MRI) 완료. 현재 완료한 객체:  2

광역동 치료술(Photodynamic therapy) 완료. 현재 완료한 객체:  170
경구 포도당 부하 검사(Oral glucose tolerance test) 완료. 현재 완료한 객체:  171
고주파 열 응고술(Radiofrequency ablation) 완료. 현재 완료한 객체:  172
전신 조사(Total body irradiation) 완료. 현재 완료한 객체:  173
색전술(Embolization) 완료. 현재 완료한 객체:  174
펀치 생검(Punch biopsy) 완료. 현재 완료한 객체:  175
감시 림프절 생검(Sentinel lymph node biopsy) 완료. 현재 완료한 객체:  176
개방 생검(Open biopsy) 완료. 현재 완료한 객체:  177
중심정맥관 삽입술(Central venous catheter insertion) 완료. 현재 완료한 객체:  178
광선요법(Phototherapy) 완료. 현재 완료한 객체:  179
복수 천자(Paracentesis) 완료. 현재 완료한 객체:  180
유두 유륜 문신 시술(Nipple tattoo) 완료. 현재 완료한 객체:  181
수혈(Tranfusion) 완료. 현재 완료한 객체:  182
추간판 조영술(Spinal discography) 완료. 현재 완료한 객체:  183
증식 치료술(Prolotherapy) 완료. 현재 완료한 객체:  184
경막외강 유착 박리술 및 신경성형술(Epidural adhesiolysis or decompressive neuroplasty) 완료. 현재 완료한 객체:  185
무릎신경 차단술(Nerve block in knee joint) 완료. 현재 완료한 객체:  186
말초신경 차단(Peripheral nerve block) 완료. 현재 완료한 객체:  187
경막외 마취(Epidural anesthesia) 완료. 현재 완료한 객체:  188
경막외 자가 혈액 봉합술(Epidural blood patch) 완료. 현재 완료

문맥대정맥지름술(Portocaval shunt operation) 완료. 현재 완료한 객체:  328
자궁부속기절제술(Adnexectomy) 완료. 현재 완료한 객체:  329
자궁사이막절제술(Uterine septectomy) 완료. 현재 완료한 객체:  330
자궁성형술(Uteroplasty) 완료. 현재 완료한 객체:  331
자궁목절제술(Cervicectomy) 완료. 현재 완료한 객체:  332
엑시머 레이저를 이용한 근시 교정 수술(Photorefractive keratectomy) 완료. 현재 완료한 객체:  333
인공 슬관절 치환 수술(Total knee replacement arthroplasty) 완료. 현재 완료한 객체:  334
인공 고관절 치환 수술(Total hip arthroplasty ) 완료. 현재 완료한 객체:  335
인공 견관절 치환 수술(Total shoulder replacement ) 완료. 현재 완료한 객체:  336
연골판 절제 수술(Arthroscopic meniscectomy) 완료. 현재 완료한 객체:  337
연골판 이식 수술(Cartilaginous plate transplantation ) 완료. 현재 완료한 객체:  338
연골판 봉합 수술(Cartilaginous plate repair) 완료. 현재 완료한 객체:  339
수근관 이완 수술(Carpal tunnel release ) 완료. 현재 완료한 객체:  340
비구 주위 절골 수술(Periacetabular osteotomy ) 완료. 현재 완료한 객체:  341
반관절성형술(Hemiarthroplasty) 완료. 현재 완료한 객체:  342
안와감압술(Orbital decompression) 완료. 현재 완료한 객체:  343
입체 정위 진공 보조 유방 생검술(Stereotactic vacuum assistant breast biopsy) 완료. 현재 완료한 객체:  344
진공 보조 유방 생검술 검사(Vacuum assistant [Vac

발기유발검사(Artificial erection test) 완료. 현재 완료한 객체:  496
방광경검사(Cystoscopy) 완료. 현재 완료한 객체:  497
방광내압측정술(Cystometry) 완료. 현재 완료한 객체:  498
요도경검사(Urethroscopy) 완료. 현재 완료한 객체:  499
요역동학검사 (Urodynamic study) 완료. 현재 완료한 객체:  500
요흐름검사(Uroflowmetry) 완료. 현재 완료한 객체:  501
음경진동각검사(Penile biothesiometry) 완료. 현재 완료한 객체:  502
역행성 요도조영술(Retrograde urethrography) 완료. 현재 완료한 객체:  503
체외수정 (In vitro fertilization ) 완료. 현재 완료한 객체:  504
태아경(Fetoscopy) 완료. 현재 완료한 객체:  505
탯줄천자(Cordocentesis) 완료. 현재 완료한 객체:  506
피부 수분 손실량 측정 검사(Transepidermal water loss) 완료. 현재 완료한 객체:  507
복막투석(Peritoneal dialysis) 완료. 현재 완료한 객체:  508
신장기능검사(Renal function test) 완료. 현재 완료한 객체:  509
혈액투석(Hemodialysis) 완료. 현재 완료한 객체:  510
식도경유심장초음파검사(Transesophageal echocardiography ) 완료. 현재 완료한 객체:  511
심장율동전환(Cardioversion) 완료. 현재 완료한 객체:  512
심장초음파검사(Echocardiography) 완료. 현재 완료한 객체:  513
약물 부하 심장 초음파 검사(Stress echocardiography) 완료. 현재 완료한 객체:  514
운동 부하 심장 초음파 검사(Exercise Stress Echocardiogram) 완료. 현재 완료한 객체:  515
폐모세혈관쐐기압(Pulmonary artery occl

In [ ]:
# http://www.amc.seoul.kr/asan/healthinfo/management/managementDetail.do?managementId=
# 확인용 테스트   검사/시술/수술 정보 0-542  -> 총 543개
i = 482
print("검사명: ", interventions[i].name)
print("관련 신체 부위: ")
for part in interventions[i].related_parts:
    print(part)
print("관련 질환 분야: ")
for area in interventions[i].related_areas:
    print(area)
print("관련 검사: ")
for intervention in interventions[i].related_interventions:
    print(intervention)
print("관련 진료과: ")
for medDep in interventions[i].medDeps:
    print(medDep)
print()
print("관련 설명: ")
for k, v in interventions[i].descrips.items():
    print(k)
    print(v)
print()
print("관련 질환 분야: ")
for dis in interventions[i].related_diseases:
    print(dis)
print("관련 인체 부위: ")
for body in interventions[i].related_bodies:
    print(body)

In [ ]:
txt= ""
for intervention in interventions:
    txt = txt + "================================= \n"
    txt = txt + "검사명: " + intervention.name + "\n"
    if len(intervention.related_parts) >= 1:
        txt = txt + "관련 신체 부위: " + "\n"
        for part in intervention.related_parts:
            txt = txt + part + "\n"
    if len(intervention.related_areas) >= 1:
        txt = txt + "관련 질환 분야: " + "\n"
        for area in intervention.related_areas:
            txt = txt + area + "\n"
    if len(intervention.related_interventions) >= 1:
        txt = txt + "관련 검사: " + "\n"
        for intervene in intervention.related_interventions:
            txt = txt + intervene + "\n"
    if len(intervention.medDeps) >= 1:
        txt = txt + "관련 진료과: " + "\n"
        for medDep in intervention.medDeps:
            txt = txt + medDep + "\n"
    if len(intervention.descrips) >= 1:
        txt = txt + "검사 설명: " + "\n"
        for k, v in intervention.descrips.items():
            txt = txt + k + "\n"
            txt = txt + v + "\n"
    if len(intervention.related_diseases) >= 1:
        txt = txt + "관련 질환: " + "\n"
        for disease in intervention.related_diseases:
            txt = txt + disease + "\n"
    if len(intervention.related_bodies) >= 1:
        txt = txt + "관련 인체: " + "\n"
        for body in intervention.related_bodies:
            txt = txt + body + "\n"
    txt = txt + "================================= \n"

with open("./output/whole/interventions.txt", "w") as text_file:
    text_file.write(txt)


In [ ]:
txt= ""
for intervention in interventions:
    txt = txt + "검사명: " + intervention.name + "\n"
    if len(intervention.descrips) >= 1:
        txt = txt + "검사 설명: " + "\n"
        for k, v in intervention.descrips.items():
            txt = txt + k + "\n"
            txt = txt + v + "\n"

with open("./output/whole/interventions_description.txt", "w") as text_file:
    text_file.write(txt)


In [ ]:
for intervention in interventions:
    if (intervention.name.find('/') != -1):
        print(intervention.name)
        intervention.name = intervention.name.replace('/','')

In [ ]:
for intervention in interventions:
    txt= ""
    txt = txt + "검사명: " + intervention.name + "\n"
    if len(intervention.descrips) >= 1:
        txt = txt + "검사 설명: " + "\n"
        for k, v in intervention.descrips.items():
            txt = txt + k + "\n"
            txt = txt + v + "\n"
    with open("./output/interventions_"+ intervention.name +".txt", "w") as text_file:
        text_file.write(txt)


# 통합 txt 파일 생성

In [ ]:
bodies_txt = open("./output/whole/bodies.txt", "r")
diseases_txt = open("./output/whole/diseases.txt", "r")
interventions_txt = open("./output/whole/interventions.txt", "r")

with open("./output/whole/whole.txt", "w") as t:
    t.write(bodies_txt.read())
    t.write(diseases_txt.read())
    t.write(interventions_txt.read())

In [ ]:
bodies_description_txt = open("./output/whole/bodies_description.txt", "r")
diseases_description_txt = open("./output/whole/diseases_description.txt", "r")
interventions_description_txt = open("./output/whole/interventions_description.txt", "r")

with open("./output/whole/whole_description.txt", "w") as t:
    bodies_string = bodies_description_txt.read()
    diseases_string = diseases_description_txt.read()
    interventions_string = interventions_description_txt.read()
    whole = bodies_string + "\n" + diseases_string + "\n" + interventions_string
    t.write(whole)

# Renaming

ETRI CLI는 파일명의 공백을 허용하지 않으므로 파일명의 공백을 모두 없애는 작업

files = os.listdir("./output/")
new_files=[]
for file in files:
    new_files.append(file.replace(" ", "_"))

for old, new in zip(files, new_files):
    os.rename("./output/"+old, "./output/"+new)

# Exporting Python objects as XML

In [66]:
#import xml.etree.ElementTree as ET
from lxml import etree as ET
from lxml.builder import E

In [118]:
bodies_xml = E("bodies")
for body in bodies:
    name = E("name", body.name)
    part = E("part", body.part)
    disease = E("disease", body.disease)
    medDep = E("medDep", body.medDep)
    
    related_diseases = E("related_diseases")
    for related_disease in body.related_diseases:
        related_diseases.append(E("related_disease", related_disease))
        
    related_interventions = E("related_interventions")
    for related_intervention in body.related_interventions:
        related_interventions.append(E("related_intervention", related_intervention))
        
    descrips = E("description")
    for k, v in body.descrips.items():
        if k == "정의":
            descrips.append(E("definition", v))
        if k == "위치":
            descrips.append(E("location", v))
        if k == "구조":
            descrips.append(E("architecture", v))
        if k == "기능":
            descrips.append(E("function", v))
        if k == "헬스팁":
            descrips.append(E("tip", v))
    
    body = E("body", name, part, disease, medDep, related_diseases, related_interventions, descrips)
    bodies_xml.append(body)

declaration = """<?xml version="1.0" encoding="utf-8" standalone="yes"?>
<!DOCTYPE bodies SYSTEM "[
<!ELEMENT body (name, part, disease, medDep, related_diseases, related_interventions, descrips)*>
<!ELEMENT name (#PCDATA)>
<!ELEMENT part (#PCDATA)>
<!ELEMENT disease (#PCDATA)>
<!ELEMENT medDep (#PCDATA)>
<!ELEMENT related_diseases (related_disease)*>
<!ELEMENT related_disease (#PCDATA)>
<!ELEMENT related_interventions (related_interventions)*>
<!ELEMENT related_intervention (#PCDATA)>
<!ELEMENT description (definition, location, architecture, function, tip)*>
<!ELEMENT definition (#PCDATA)>
<!ELEMENT location (#PCDATA)>
<!ELEMENT architecture (#PCDATA)>
<!ELEMENT function (#PCDATA)>
<!ELEMENT tip (#PCDATA)>
]" >
<!--Seongtae Kim 2020-06-06 https://github.com/Seongtae-Kim -->
"""

dec_byte = str.encode(declaration)
bodies_tree = ET.ElementTree(bodies_xml)
end_com= str.encode("<!--Seongtae Kim 2020-06-06 https://github.com/Seongtae-Kim -->")

with open("./bodies.xml", "wb") as f:
    f.write(dec_byte)
    f.write(ET.tostring(bodies_tree, xml_declaration=False, encoding="utf-8", pretty_print=True))
    f.write(end_com)

In [119]:
diseases_xml = E("diseases")
for disease in diseases:
    name = E("name", disease.name)
    synonym = E("synonym", disease.synonym)
    
    related_diseases = E("related_diseases")
    for related_disease in disease.related_diseases:
        related_diseases.append(E("related_disease", related_disease))
        
    related_symptoms = E("related_symptoms")
    for related_symptom in disease.related_symptoms:
        related_symptoms.append(E("related_symptom", related_symptom))
        
    medDeps = E("medDeps")
    for medDep in disease.medDeps:
        medDeps.append(E("medDep", medDep))
        
    descrips = E("description")
    for k, v in disease.descrips.items():
        if k == "정의":
            descrips.append(E("definition", v))
        if k == "원인":
            descrips.append(E("cause", v))
        if k == "증상":
            descrips.append(E("symptom", v))
        if k == "진단":
            descrips.append(E("diagnosis", v))
        if k == "경과":
            descrips.append(E("progress", v))
        if k == "주의사항":
            descrips.append(E("caution", v))
    
    disease = E("disease", name, related_diseases, related_symptoms, medDeps, synonym, descrips)
    diseases_xml.append(disease)

declaration = """<?xml version="1.0" encoding="utf-8" standalone="yes"?>
<!DOCTYPE diseases SYSTEM "[
<!ELEMENT disease (name, related_diseases, related_symptoms, medDeps, synonym, descrips)*>
<!ELEMENT name (#PCDATA)>
<!ELEMENT synonym (#PCDATA)>
<!ELEMENT related_diseases (related_disease)*>
<!ELEMENT related_disease (#PCDATA)>
<!ELEMENT related_symptoms (related_symptoms)*>
<!ELEMENT related_symptom (#PCDATA)>
<!ELEMENT medDeps (medDeps)*>
<!ELEMENT medDep (#PCDATA)>
<!ELEMENT description (definition, cause, symptom, diagnosis, progress, caution)*>
<!ELEMENT definition (#PCDATA)>
<!ELEMENT cause (#PCDATA)>
<!ELEMENT symptom (#PCDATA)>
<!ELEMENT diagnosis (#PCDATA)>
<!ELEMENT progress (#PCDATA)>
<!ELEMENT caution (#PCDATA)>
]">
<!--Seongtae Kim 2020-06-06 https://github.com/Seongtae-Kim -->
"""
dec_byte = str.encode(declaration)
end_com= str.encode("<!--Seongtae Kim 2020-06-06 https://github.com/Seongtae-Kim -->")
diseases_tree = ET.ElementTree(diseases_xml)
with open("./diseases.xml", "wb") as f:
    f.write(dec_byte)
    f.write(ET.tostring(diseases_tree, xml_declaration=False, encoding="utf-8", pretty_print=True))
    f.write(end_com)

In [120]:
interventions_xml = E("interventions")
for intervention in interventions:
    name = E("name", intervention.name)
    
    related_diseases = E("related_diseases")
    for related_disease in intervention.related_diseases:
        related_diseases.append(E("related_disease", related_disease))
        
    related_bodies = E("related_bodies")
    for related_body in intervention.related_bodies:
        related_bodies.append(E("related_body", related_body))
        
    related_parts = E("related_parts")
    for related_part in intervention.related_parts:
        related_parts.append(E("related_part", related_part))
        
    related_areas = E("related_areas")
    for related_area in intervention.related_areas:
        related_areas.append(E("related_area", related_area))
        
    related_interventions = E("related_interventions")
    for related_intervention in intervention.related_interventions:
        related_interventions.append(E("related_intervention", related_intervention))
        
    medDeps = E("medDeps")
    for medDep in intervention.medDeps:
        medDeps.append(E("medDep", medDep))
        
    descrips = E("description")
    for k, v in intervention.descrips.items():
        if k == "정의":
            descrips.append(E("definition", v))
        if k == "적응증":
            descrips.append(E("indication", v))
        if k == "준비사항":
            descrips.append(E("preparation", v))
        if k == "검사시기":
            descrips.append(E("stage", v))
        if k == "검사/시술/수술 방법":
            descrips.append(E("method", v, encoding="unicode"))
        if k == "검사결과":
            descrips.append(E("result", v, encoding="unicode"))
        if k == "주의사항":
            descrips.append(E("caution", v, encoding="unicode"))
        if k == "헬스팁":
            descrips.append(E("tip", v, encoding="unicode"))
    
    intervention = E("intervention", name, related_parts, related_areas, related_interventions, related_diseases, related_bodies, medDeps, descrips)
    interventions_xml.append(intervention)

declaration = """<?xml version="1.0" encoding="utf-8" standalone="yes"?>
<!DOCTYPE interventions SYSTEM "[
<!ELEMENT intervention (name, related_parts, related_areas, related_interventions, related_diseases, related_bodies, medDeps, descrips)*>
<!ELEMENT name (#PCDATA)>
<!ELEMENT related_diseases (related_disease)*>
<!ELEMENT related_disease (#PCDATA)>
<!ELEMENT related_bodies (related_bodies)*>
<!ELEMENT related_body (#PCDATA)>
<!ELEMENT related_parts (related_parts)*>
<!ELEMENT related_part (#PCDATA)>
<!ELEMENT related_areas (related_areas)*>
<!ELEMENT related_area (#PCDATA)>
<!ELEMENT related_interventions (related_interventions)*>
<!ELEMENT related_intervention (#PCDATA)>
<!ELEMENT medDeps (medDeps)*>
<!ELEMENT medDep (#PCDATA)>
<!ELEMENT description (definition, indication, preparation, stage, method, result, caution, tip)*>
<!ELEMENT definition (#PCDATA)>
<!ELEMENT indication (#PCDATA)>
<!ELEMENT symptom (#PCDATA)>
<!ELEMENT preparation (#PCDATA)>
<!ELEMENT stage (#PCDATA)>
<!ELEMENT method (#PCDATA)>
<!ELEMENT result (#PCDATA)>
<!ELEMENT caution (#PCDATA)>
<!ELEMENT tip (#PCDATA)>
]">
<!--Seongtae Kim 2020-06-06 https://github.com/Seongtae-Kim -->
"""

dec_byte = str.encode(declaration)
end_com= str.encode("<!--Seongtae Kim 2020-06-06 https://github.com/Seongtae-Kim -->")
interventions_tree = ET.ElementTree(interventions_xml)
with open("./interventions.xml", "wb") as f:
    f.write(dec_byte)
    f.write(ET.tostring(interventions_tree, encoding="utf-8", pretty_print=True))
    f.write(end_com)

In [121]:
declaration = """<?xml version="1.0" encoding="utf-8" standalone="yes"?>
<!DOCTYPE seoulAsan SYSTEM "[
<!ELEMENT bodies (body)>
<!ELEMENT diseases (disease)>
<!ELEMENT interventions (intervention)>

<!ELEMENT body (name, part, disease, medDep, related_diseases, related_interventions, descrips)*>
<!ELEMENT name (#PCDATA)>
<!ELEMENT part (#PCDATA)>
<!ELEMENT disease (#PCDATA)>
<!ELEMENT medDep (#PCDATA)>
<!ELEMENT related_diseases (related_disease)*>
<!ELEMENT related_disease (#PCDATA)>
<!ELEMENT related_interventions (related_interventions)*>
<!ELEMENT related_intervention (#PCDATA)>
<!ELEMENT description (definition, location, architecture, function, tip)*>
<!ELEMENT definition (#PCDATA)>
<!ELEMENT location (#PCDATA)>
<!ELEMENT architecture (#PCDATA)>
<!ELEMENT function (#PCDATA)>
<!ELEMENT tip (#PCDATA)>

<!ELEMENT disease (name, related_diseases, related_symptoms, medDeps, synonym, descrips)*>
<!ELEMENT name (#PCDATA)>
<!ELEMENT synonym (#PCDATA)>
<!ELEMENT related_diseases (related_disease)*>
<!ELEMENT related_disease (#PCDATA)>
<!ELEMENT related_symptoms (related_symptoms)*>
<!ELEMENT related_symptom (#PCDATA)>
<!ELEMENT medDeps (medDeps)*>
<!ELEMENT medDep (#PCDATA)>
<!ELEMENT description (definition, cause, symptom, diagnosis, progress, caution)*>
<!ELEMENT definition (#PCDATA)>
<!ELEMENT cause (#PCDATA)>
<!ELEMENT symptom (#PCDATA)>
<!ELEMENT diagnosis (#PCDATA)>
<!ELEMENT progress (#PCDATA)>
<!ELEMENT caution (#PCDATA)>

<!ELEMENT intervention (name, related_parts, related_areas, related_interventions, related_diseases, related_bodies, medDeps, descrips)*>
<!ELEMENT name (#PCDATA)>
<!ELEMENT related_diseases (related_disease)*>
<!ELEMENT related_disease (#PCDATA)>
<!ELEMENT related_bodies (related_bodies)*>
<!ELEMENT related_body (#PCDATA)>
<!ELEMENT related_parts (related_parts)*>
<!ELEMENT related_part (#PCDATA)>
<!ELEMENT related_areas (related_areas)*>
<!ELEMENT related_area (#PCDATA)>
<!ELEMENT related_interventions (related_interventions)*>
<!ELEMENT related_intervention (#PCDATA)>
<!ELEMENT medDeps (medDeps)*>
<!ELEMENT medDep (#PCDATA)>
<!ELEMENT description (definition, indication, preparation, stage, method, result, caution, tip)*>
<!ELEMENT definition (#PCDATA)>
<!ELEMENT indication (#PCDATA)>
<!ELEMENT symptom (#PCDATA)>
<!ELEMENT preparation (#PCDATA)>
<!ELEMENT stage (#PCDATA)>
<!ELEMENT method (#PCDATA)>
<!ELEMENT result (#PCDATA)>
<!ELEMENT caution (#PCDATA)>
<!ELEMENT tip (#PCDATA)>
]">
<!--Seongtae Kim 2020-06-06 https://github.com/Seongtae-Kim -->
"""
whole_xml = E("seoulAsan", bodies_xml, diseases_xml, interventions_xml, encoding="unicode")
whole_tree = ET.ElementTree(whole_xml)
dec_byte = str.encode(declaration)
end_com = str.encode("<!--Seongtae Kim 2020-06-06 https://github.com/Seongtae-Kim -->")

with open("./SeoulAsanData.xml", "wb") as f:
    f.write(dec_byte)
    f.write(ET.tostring(whole_tree, encoding="utf-8", pretty_print=True))
    f.write(end_com)